#### Batch Data Ingestion & Preparation

##### https://www.citibikenyc.com/system-data

In [2]:
from pyspark.sql.types import *
from pyspark.sql import functions as psf

In [3]:
# Config section:
bronze_path = '/FileStore/tables/Citi-Bike/Bronze'
silver_path = '/FileStore/tables/Citi-Bike/Silver'
gold_path = '/FileStore/tables/Citi-Bike/Gold'

dbutils.fs.mkdirs(bronze_path)
dbutils.fs.mkdirs(silver_path)
dbutils.fs.mkdirs(gold_path)

###### Hive like Catalog DB named: citibike to store Project Tables & Views:

In [5]:
%sql
DROP DATABASE IF EXISTS citibike CASCADE;
CREATE DATABASE citibike;

In [6]:
display(
  dbutils.fs.ls(bronze_path)
)

In [7]:
# View and Delete unwanted files from Landing/Bronze directory:
dbutils.fs.ls('/FileStore/tables/')
#dbutils.fs.rm('/FileStore/tables/JC_202006_citibike_tripdata.csv')

In [8]:
# schema for the raw trip history data
trip_schema = StructType([
  StructField('tripduration', IntegerType()),
  StructField('start_time', StringType()),
  StructField('stop_time',  StringType()),
  StructField('start_station_id', IntegerType()),
  StructField('start_station_name', StringType()),
  StructField('start_station_latitude', StringType()),
  StructField('start_station_longitude', StringType()),
  StructField('end_station_id', IntegerType()),
  StructField('end_station_name', StringType()),
  StructField('end_station_latitude', StringType()),
  StructField('end_station_longitude', StringType()),
  StructField('bike_id', IntegerType()),
  StructField('user_type', StringType()),
  StructField('birth_year', StringType()),
  StructField('user_gender', StringType()),
  ])

In [9]:
# read the raw trip history data to dataframe, without triggering job, by passing csv schema
raw = spark.read.csv(
  bronze_path, 
  header=True,  
  schema=trip_schema
  )

In [10]:
#raw.rdd.getNumPartitions()

In [11]:
#raw = raw.coalesce(200) # If we use Coalesce to reduce NumPartitions, then we can use Coalesce to INCREASE numPart till that old num !!
#raw = raw.repartition(201, ["start_station_id", "end_station_id"])
#raw.rdd.getNumPartitions()

In [12]:
raw = raw.distinct()
#display(raw.orderBy(raw.start_time.desc()))
#raw.distinct().count() #101046
raw.count()  #101046

##### Complete Refresh / Append to Delta Lake - Silver Layer

In [14]:
# Save combined de-duped CSVs as Delta Lake in Silver layer DBFS:
raw.write.format('delta').mode('overwrite').save(silver_path)

In [15]:
#Verify write to Silver:
slvrDF = spark.read.format('delta').load(silver_path)

In [16]:
print("slvrDF.count()  :   ", slvrDF.count())
print("slvrDF.dtypes  :   ", slvrDF.dtypes)
print("slvrDF.head()  :   ", slvrDF.head())
print("slvrDF.schema  :   ", slvrDF.schema)

In [17]:
display(slvrDF.head(10))

##### Create a Station Master Dimension table

In [19]:
slvrDF.createOrReplaceTempView("silver")

startDF = slvrDF.select(["start_station_id","start_station_name"]).distinct().withColumnRenamed("start_station_id","station_id").withColumnRenamed("start_station_name","station_name")
endDF = slvrDF.select(["end_station_id","end_station_name"]).distinct().withColumnRenamed("end_station_id","station_id").withColumnRenamed("end_station_name","station_name")
unionDF = startDF.unionAll(endDF).distinct()

#display(unionDF)

unionDF.write.format('delta').mode('overwrite').save(f"{gold_path}/dim_station")

In [20]:
# Create Managed Table using spark SQL (alternative to Hive SQL)
tableName = "dim_station"

spark.sql(f"DROP TABLE IF EXISTS {tableName}")
spark.sql(f"DROP TABLE IF EXISTS CITIBIKE.{tableName}")

spark.sql(f"""
  CREATE TABLE CITIBIKE.{tableName}
  USING DELTA
  LOCATION "{gold_path}/dim_station"
""")

In [21]:
spark.catalog.listTables()

In [22]:
%sql
select * from CITIBIKE.dim_station

#### Perform Top N Analysis - Gold Layer:
* Busiest Station-ID: Stations with most Trips
* Stations with Longest Trips
* Stations with Shortest Trips
* Customers by Age group : 30+, 40+, 50+ yrs 
* Stations with Most Subscribers

###### Spark SQL approach:
###### Busiest Station-ID: Stations with most Trips

In [25]:
#Busiest Station-ID: Stations with most Trips
slvrDF.createOrReplaceTempView("silver")

#stations_by_cnt = 
spark.sql('''
select station_id, count(station_id) AS trip_cnt
FROM 
  (
     select start_station_id as station_id from silver
     UNION ALL
     select end_station_id as station_id from silver
  ) AS station_ids
GROUP BY station_id
order by trip_cnt DESC
'''
).createOrReplaceTempView("dm_busiest_stations")

#display(stations_by_cnt) # IF a Dataframe was created, then we have seen Content this way

In [26]:
%sql
select A.station_id, S.station_name, A.trip_cnt
from dm_busiest_stations A
INNER JOIN CITIBIKE.dim_station S
on A.station_id = S.station_id

##### Stations with Longest Trips

In [28]:
%sql
-- MEDIAN driven approach:
select DISTINCT A.*, S.station_name FROM (
  select start_station_id as station_id, tripduration, PERCENT_RANK() OVER (PARTITION BY start_station_id ORDER BY tripduration) AS median_tripduration
  from silver
  UNION ALL
  select end_station_id as station_id, tripduration, PERCENT_RANK() OVER (PARTITION BY start_station_id ORDER BY tripduration) AS median_tripduration
  from silver
) A
INNER JOIN CITIBIKE.dim_station S
on A.station_id = S.station_id
WHERE A.median_tripduration = 0.5
ORDER BY A.tripduration DESC, A.station_id
LIMIT 10

In [29]:
%sql
-- MEAN/AVG based approach:
select A.start_station_id, round(avg(A.tripduration),0) as avg_tripduration, S.station_name
from silver A
INNER JOIN CITIBIKE.dim_station S
on A.start_station_id = S.station_id
GROUP BY A.start_station_id, S.station_name
--order by tripduration DESC

In [30]:
%sql
-- 1st Draft SQL approach:
-- Spark SQL approach:
select start_station_id, end_station_id, tripduration
from silver
order by tripduration DESC

In [31]:
# Verified tripduraion column value - by deriving the duration here itself:
display(slvrDF.withColumn("trip_duration_sec", psf.unix_timestamp("stop_time") - psf.unix_timestamp("start_time")).orderBy("tripduration", ascending=0))

##### REF: sample code from DB https://databricks.com/notebooks/recitibikenycdraft/data-preparation.html
##### Below code not to be run as part of this pipeline:

In [33]:
# cleanse the data in preparation for analysis
cleansed = (
  raw
    .select(
      raw.start_station_id.alias('station_id'),
      psf.coalesce( 
        psf.to_timestamp(raw.start_time, 'yyyy-MM-dd HH:mm:SS'),   # most files use this datetime format
        psf.to_timestamp(raw.start_time, 'MM/dd/yyyy HH:mm:SS'),   # some 2015 files use this datetime format
        psf.to_timestamp(raw.start_time, 'MM/dd/yyyy HH:mm')       # some 2015 files use this datetime format
        ).alias('start_time')
      )
    .filter('(station_id Is Not Null) AND (start_time Is Not Null)') # remove any bad records
  )
cleansed.createOrReplaceTempView('trips')

display(cleansed)

In [34]:
# this ensures we don't duplicate records on repeated runs of this notebook
try:
  dbutils.fs.rm('/mnt/citibike/rentals', recurse=True)
except:
  pass

In [35]:
%sql
DROP TABLE IF EXISTS citibike.rentals;

CREATE TABLE citibike.rentals(
  station_id INTEGER,
  hour TIMESTAMP,
  rentals INTEGER
  )
  USING delta
  LOCATION '/mnt/citibike/rentals';
  
INSERT INTO citibike.rentals
  SELECT
    x.station_id,
    x.hour,
    COUNT(*) as rentals
  FROM ( 
    SELECT
      station_id,
      CAST( -- truncate start time to the hour
        CONCAT(
          CAST(DATE(start_time) as string), ' ',
          EXTRACT(hour from start_time), ':00:00.000'
          ) as TIMESTAMP
        ) as hour
      FROM trips
    ) x
    GROUP BY x.station_id, x.hour;  

In [36]:
%sql
select * from citibike.rentals
order by hour DESC

In [37]:
v = spark.sql("select * from citibike.rentals \
          order by hour DESC")
display(v)

In [38]:
display(
  dbutils.fs.ls('/FileStore/tables/Citi-Bike/api_response')
)

#dbutils.fs.rm('/FileStore/tables/Citi-Bike/api_response/station_status_2020062514-22-51.json')

In [39]:
%sh
cat /dbfs/FileStore/tables/Citi-Bike/api_response/station_status_20200625_14-25-07.json